# Use Case Generation Program


### generate usecases by searching the different data library for interested parameters and output a UseCase_set with the correct format for the automation test. 

#### Excel file: Data_Library.xlsx

#### Libraries: 
- Data with eZono machine
- Data with Toshiba machine
- Data with GE machine
- Data with SonoSite machine
- Tingting's 3D data with SonoSite machine

#### Parameters (column names):
DataFolderName	StudyName	Owner	Medium	AutomaticTankTest	ManualTankTest	US_machine	NeedleType	NeedleID	TransducerLength_mm	NeedleLength_in	PicoSeries	SamplingRate_MSps	SampleInterval_ns	SampleNum	GainSetting_dB	TestType filename	Axis0	Axis1	Axis2	Axis3	Axis4

In [133]:
import pandas as pd
import numpy as np
import os
from openpyxl import load_workbook

def create_usecase(data_df_list, usecase_df, lib_names, col_names, col_values): 
    """
    Goes through each library to search for the given column name and values to create a usecase information data frame.
    
    data_df_list: list of data frame objects, ea. data frame is data from 1 library
    usecase_df: data frame object of TestUseCase sheet
    lib_names: list of library names to search through. ex: ['Lib1', 'Lib2']
    col_names: list of column names to search. ex: ['Axis0', 'Axis1']
    col_values: nested list, with ea sublist contains value conditions. ex: [[60,60],[0,0]]
    return: a data frame of the usecase created
    """
    row_num = usecase_df.shape[0]+1  # next row in the TestUseCase sheet
    usecase_format=[]
    for lib in range(len(lib_names)):  # for each library
        
        # filtering normal data:
        n= [c for c in col_names if not c.startswith('Axis')]
        v= [col_values[i] for i in range(len(col_names)) if not col_names[i].startswith('Axis') ]
        filtered_lib= data_df_list[lib]
        for i,item in enumerate(n): 
            filtered_lib= filtered_lib[filtered_lib[item].isin(v[i])]
        
        # filtering axis data which can have multiple values:        
        signal_list=[]
        axiscol= [c for c in col_names if c.startswith('Axis')]
        axisval= [col_values[i] for i in range(len(col_names)) if col_names[i].startswith('Axis')]
       
        #filter through each set of coordinates:
        for a in range(len(axisval[0])):
            lib_df= filtered_lib
            for p in range(len(axisval)): # filter through each condition:
                condition=axisval[p][a]
                axis= axiscol[p]
                lib_df= lib_df[lib_df[axis].isin([condition])]
            signal_list= signal_list+ list(lib_df['Row'].values)    

        if signal_list==[]:continue     
        # format use case string: 
        signal_str= ",".join(str(s) for s in signal_list)  # ex: '23,45,65'
        if signal_str != '' or signal_str != ' ':  # add to list if signal is not empty
            usecase_format += ["{}: ({})".format(lib_names[lib], signal_str)]

    usecase_format= ";".join(str(s) for s in usecase_format)  # convert from list of strings to 1 string
    generated_usecase_df = pd.DataFrame([[row_num,usecase_format]],columns = ["Row","UseCase_set"])
    return generated_usecase_df

def main(): 
    """
    Prompt for user input:
        1) excel file name - if none given, default file will be used
        2) input row number - row number from UseCaseGenerationInput to load
        3) library - name of library to use
        4) usecase description - if none given, default is a string of 'Column name = condition'. ex: 'Axis0'=0, 'Axis2'=1
    
    Note: Axis are allowed multiple values for usecases where needle is moving around (trajectory or in-out of plane)
    Make sure all axis has the same number of coordinates even if they are repeating (ex: 0,0,0,0)
    
    """
    # STEP 1: PROMPT FOR USER INPUT

    # input name of excel file
    default_file= 'RX_Data_Library.xlsx'
    filename= input('Enter the data file name or press ENTER to use default file: ')
    if filename=='' or not os.path.isfile(filename): 
        print('Default data file {} will be used.\n'.format(default_file))
        filename= default_file

    # input rule number
    ruleNum= int(input('Enter the use case input row number: '))    
    usecasegeninput= pd.read_excel(filename, sheet_name= 'UseCaseGenerationInput')
    while ruleNum > max(usecasegeninput['Rule_Number']): # check if rule number exist
        print('Error: Input Row Number does not exist')
        ruleNum= int(input('Enter the use case input row number: ')) 
    input_df= usecasegeninput.iloc[ruleNum-1]
    
    # input the Library 
    summary_df = pd.read_excel(filename, sheet_name= 'Summary') 
    lib_avail = set(summary_df['LibraryName'].unique())
    print("\nAvailable library: {}\n".format(lib_avail))
    done=False
    while done== False:
        lib_input= input('Enter the library numbers, separated by comma: ') 
        lib_input= lib_input.replace(' ', '').replace('lib','').split(',')
        lib_input= set(['Lib'+name for name in lib_input])
        lib_names= list(lib_input.intersection(lib_avail)) #matching lib_input with lib_avail
        if lib_names!=[]:         
            print("\nLibrary use: {}\n".format(lib_names)) 
            done=True
        else: 
            print("\nError: No Library Found.\n") 
                
    # input description
    description= input('Enter use case description or ENTER to use default: ')

    # STEP 2: EXTRACT INPUT INFORMATION
    
    print("\nExtracting input information...\n")
    # get column names
    input_series = input_df[input_df.notnull()] #pandas series
    col_names = [a for a in input_series.index if 'Rule_Number' not in a]  # all column names with conditions

    # get column values
    axis_names = [a for a in col_names if a.startswith('Axis')]  # Axis columns
    col_values = [[input_series[col]] for col in col_names if not col.startswith('Axis')]  
    axis_values=[]
    for axis in axis_names: 
        if not type(input_series[axis])== int: 
            cond_set= [int(a) for a in input_series[axis].split(',')]  # 1 set of conditions for each axis
        else: 
            cond_set= [str(input_series[axis])]
        axis_values = axis_values + [cond_set] 
        col_values = col_values + [cond_set] # add to values list
    
    # check if axis have the same number of values
    if len(np.unique(np.array([len(a) for a in axis_values])))!= 1: 
        print('Error: Input Axis values have length mismatch. Check Axis values assigned in Rule Number {}.'.format(ruleNum))
        return []
    
    # STEP 3: IMPORT DATA FROM DATALIBRARY
    
    data_df_list=[] #list of library dataframes
    alldata_df= pd.read_excel(filename,sheet_name = 'DataLibrary')
    for lib in lib_names:
        data_df_list += [alldata_df[alldata_df['LibraryName']==lib]] # list of dataFrame objects
    usecase_df = pd.read_excel(filename,sheet_name = 'TestUseCase')  
    usecase_df.dropna(axis=0, how='all')  # drop any empty row 
    
    # STEp 4: ASSIGN CONDITIONS AND CREATE USECASES
    
    print("Creating use case... \n")
    usecase= create_usecase(data_df_list, usecase_df, lib_names,col_names, col_values)
    if description== '':
        usecase['Description']= '; '.join(['{}={}'.format( col_names[s], str(col_values[s]).strip('[]')) for s in range(len(col_values))])
    else:
        usecase['Description']= description
    usecase['Rule_Number']= ruleNum    
    usecase['Lib_Used']= str(lib_names)
    
    # STEP 5: EXPORT DATA
    
    # save to same file: 
    book = load_workbook(filename) # open excel workbook
    sheets= book.sheetnames 
    ws= book[sheets[sheets.index('TestUseCase')]] # TestUseCase sheet
    for index, row in usecase.iterrows():    
        ws.append(row.tolist()) # add new usecase
    book.save(filename)

    print('Done! New usecase is added and saved in the same file: {}'.format(filename))
    return usecase

## Example 1: Creating a simple Usecase

Gives conditions for certain columns in each library and look for data that matched the conditions

    Input is Row #1 from UseCaseGnerationInput sheet. 
    Search library 3,4 for 0.6mm PZT in Tissue at 20mm away from probe and 45 deg insertion angle.

In [115]:
df= main()
df

Enter the data file name or press ENTER to use default file: 
Default data file RX_Data_Library.xlsx will be used.

Enter the use case input row number: 1

Available library: ['Lib1' 'Lib2' 'Lib3' 'Lib4' 'Lib5']

Enter the library names or numbers, separated by comma: 3,4

Library use: ['Lib3', 'Lib4']

Enter use case description or ENTER to use default: 

Extracting input information...

Creating use case... 

Done! New usecase is added and saved in the same file: RX_Data_Library.xlsx


,Row,UseCase_set,Description,Rule_Number,Lib_Used
0,5,"Lib3: (115,123,124,1108,1116,1117);Lib4: (179,...",Medium='Tissue'; TransducerLength_mm=0.6; Axis...,1,"['Lib3', 'Lib4']"


## Example 2: Creating Trajectory

Searching the different data library for data where needle inserting (45 degrees angle insertion) starting from the upper left edge of the ultrasound machine to a lower depth toward the middle of the ultrasound machine. 

    Input is Row #2 from UseCaseGenerationInput sheet
    Search Lib1
    Axis 2: at 0,10,18 mm, where 0 is the middle of the US screen and 18 is the edge of the screen
    Axis 0: at 20,30,40
    Axis 3: 45 degrees insertion

In [116]:
df= main()
df

Enter the data file name or press ENTER to use default file: 
Default data file RX_Data_Library.xlsx will be used.

Enter the use case input row number: 2

Available library: ['Lib1' 'Lib2' 'Lib3' 'Lib4' 'Lib5']

Enter the library names or numbers, separated by comma: 1

Library use: ['Lib1']

Enter use case description or ENTER to use default: 

Extracting input information...

Creating use case... 

Done! New usecase is added and saved in the same file: RX_Data_Library.xlsx


,Row,UseCase_set,Description,Rule_Number,Lib_Used
0,5,"Lib1: (219,248,303,332,4,14,24,34,43,221,250,3...","Axis0=20, 30, 40; Axis1=0, 0, 0; Axis2=18, 0, ...",2,['Lib1']


## Example 3: Creating In and Out of Plane Usecase
using data from Lib5: Tingting's 3D tank data with SonoSite

    Input is Row #3 from UseCaseGenerationInput sheet
    Lib5 
    Axis0: at 50mm away from probe
    Axis1: at 0,10,20mm away from center line

In [117]:
df= main()
df

Enter the data file name or press ENTER to use default file: 3
Default data file RX_Data_Library.xlsx will be used.

Enter the use case input row number: 3

Available library: ['Lib1' 'Lib2' 'Lib3' 'Lib4' 'Lib5']

Enter the library names or numbers, separated by comma: 5

Library use: ['Lib5']

Enter use case description or ENTER to use default: 

Extracting input information...

Creating use case... 

Done! New usecase is added and saved in the same file: RX_Data_Library.xlsx


,Row,UseCase_set,Description,Rule_Number,Lib_Used
0,6,"Lib5: (3324,3334,3344)","Axis0=50, 50, 50; Axis1=0, 0, 0; Axis2=0, 10, 20",3,['Lib5']


##  Example 4: Creating Trajectory from 3D coordinates
    
    Needle 0 degree insertion angle, moving form point 1 to point 6 in a straight line toward the mid line.
    
    Input is Row #4 from UseCaseGenerationInput sheet
    Lib5
    Axis0: 10,20,30,40,50,60 mm away from probe
    Axis1: 0
    Axis2: 20,16,12,8,4,0    

In [118]:
df= main()
df

Enter the data file name or press ENTER to use default file: 
Default data file RX_Data_Library.xlsx will be used.

Enter the use case input row number: 4

Available library: ['Lib1' 'Lib2' 'Lib3' 'Lib4' 'Lib5']

Enter the library names or numbers, separated by comma: 5

Library use: ['Lib5']

Enter use case description or ENTER to use default: 

Extracting input information...

Creating use case... 

Done! New usecase is added and saved in the same file: RX_Data_Library.xlsx


,Row,UseCase_set,Description,Rule_Number,Lib_Used
0,7,"Lib5: (1228,1753,2278,2803,3328,3853)","Axis0=10, 20, 30, 40, 50, 60; Axis1=0, 0, 0, 0...",4,['Lib5']
